In [1]:
!pip install --upgrade pip
!pip install pandas lxml html5lib beautifulsoup4 requests openpyxl 

In [2]:
import pandas as pd
import re
from datetime import datetime

# URL страницы Википедии со списком компаний S&P 500
URL_WIKIPEDIA_SP500 = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'

print(f"Попытка загрузить таблицы с: {URL_WIKIPEDIA_SP500}")

try:
    # Загружаем все таблицы с HTML-страницы
    tables = pd.read_html(URL_WIKIPEDIA_SP500)
    
    if not tables:
        print("На странице не найдено таблиц.")
    else:
        print(f"Найдено {len(tables)} таблиц. Предполагаем, что первая таблица содержит список S&P 500.")
        
        # Обычно первая таблица на странице - это список компонентов S&P 500
        sp500_df = tables[0].copy() # Используем .copy() для работы с копией
        
        print("\nСтолбцы в загруженной таблице:")
        print(sp500_df.columns)
        
        # --- Шаг 1: Создание DataFrame с тикерами, именами и годом добавления ---
        
        # Определение имен столбцов (могут немного отличаться на Википедии)
        symbol_col = 'Symbol'
        name_col = 'Security' # Название компании
        
        # Поиск столбца с датой добавления
        date_added_col = None
        possible_date_cols = ['Date added', 'Date first added', 'First added']
        for col_name in possible_date_cols:
            if col_name in sp500_df.columns:
                date_added_col = col_name
                break
        
        if not date_added_col:
            raise ValueError(f"Не удалось найти столбец с датой добавления. Проверьте столбцы: {sp500_df.columns}")

        print(f"\nИспользуемые столбцы: Тикер='{symbol_col}', Имя='{name_col}', Дата добавления='{date_added_col}'")

        # Выбор нужных столбцов
        if symbol_col not in sp500_df.columns or name_col not in sp500_df.columns:
            raise ValueError(f"Не найдены необходимые столбцы '{symbol_col}' или '{name_col}'. Проверьте столбцы: {sp500_df.columns}")
            
        df_companies = sp500_df[[symbol_col, name_col, date_added_col]].copy()
        df_companies.rename(columns={
            symbol_col: 'Ticker',
            name_col: 'Name',
            date_added_col: 'DateAddedRaw'
        }, inplace=True)

        # Функция для извлечения года из строки с датой
        def extract_year_from_date(date_entry):
            if pd.isna(date_entry):
                return None
            date_str = str(date_entry)
            # Сначала пытаемся распознать полные даты (например, "1999-06-30", "June 30, 1999")
            try:
                # pandas to_datetime может обработать много форматов
                dt_object = pd.to_datetime(date_str, errors='coerce')
                if pd.notna(dt_object):
                    return dt_object.year
            except Exception:
                pass # Если не получилось, пробуем регулярное выражение
            
            # Если to_datetime не справился или вернул NaT, ищем 4 цифры года
            match = re.search(r'(\b\d{4}\b)', date_str)
            if match:
                return int(match.group(1))
            return None

        df_companies['YearAdded'] = df_companies['DateAddedRaw'].apply(extract_year_from_date)
        
        # Удаляем строки, где год извлечь не удалось
        df_companies.dropna(subset=['YearAdded'], inplace=True)
        df_companies['YearAdded'] = df_companies['YearAdded'].astype(int)

        print("\nDataFrame с извлеченными годами (первые 5 строк):")
        print(df_companies.head())

        # --- Шаг 2: Извлечение года и подсчет количества акций, добавленных каждый год ---
        additions_per_year = df_companies.groupby('YearAdded').size().reset_index(name='NumberOfAdditions')
        
        print("\nКоличество акций, добавленных по годам (из текущего списка):")
        print(additions_per_year.sort_values(by='NumberOfAdditions', ascending=False).head(10))

        # --- Шаг 3: Определение года с наибольшим количеством добавлений (исключая 1957) ---
        # Исключаем 1957 год, так как это год основания индекса в его текущем формате
        additions_filtered = additions_per_year[additions_per_year['YearAdded'] != 1957]
        
        if not additions_filtered.empty:
            max_additions_count = additions_filtered['NumberOfAdditions'].max()
            years_with_max_additions = additions_filtered[additions_filtered['NumberOfAdditions'] == max_additions_count]
            
            # Если несколько лет с одинаковым максимумом, выбираем самый последний
            year_highest_additions = years_with_max_additions['YearAdded'].max()
            
            print(f"\n--------------------------------------------------------------------")
            print(f"Вопрос 1: Год с наибольшим количеством добавлений (исключая 1957): {year_highest_additions}")
            print(f"           (Количество добавлений в этот год: {max_additions_count})")
            print(f"--------------------------------------------------------------------")
        else:
            print("\nНе удалось определить год с наибольшим количеством добавлений после фильтрации.")

        # --- Дополнительный вопрос: Сколько текущих акций S&P 500 находятся в индексе более 20 лет? ---
        current_year = datetime.now().year
        threshold_year = current_year - 21 # Более 20 лет означает добавление за 21 год до текущего или ранее
                                          # Например, если сейчас 2024, то "более 20 лет" = 2024 - 21 = 2003 и ранее.
                                          # (2024-2003 = 21 год)

        stocks_over_20_years = df_companies[df_companies['YearAdded'] <= threshold_year]
        count_stocks_over_20_years = len(stocks_over_20_years)
        
        print(f"\n--------------------------------------------------------------------")
        print(f"Дополнительно: Количество текущих акций S&P 500, находящихся в индексе")
        print(f"              более 20 лет (добавлены в {threshold_year} году или ранее): {count_stocks_over_20_years}")
        print(f"--------------------------------------------------------------------")
        
        # Показать примеры таких акций
        print("\nПримеры акций, находящихся в индексе более 20 лет (первые 5):")
        print(stocks_over_20_years[['Ticker', 'Name', 'YearAdded']].head())


except requests.exceptions.RequestException as e:
    print(f"\nСетевая ошибка при попытке доступа к {URL_WIKIPEDIA_SP500}: {e}")
    print("Пожалуйста, проверьте ваше интернет-соединение.")
except ValueError as e:
    print(f"\nОшибка значения или обработки данных: {e}")
    print("Это может быть связано с изменениями в структуре таблицы на Википедии.")
except Exception as e:
    print(f"\nПроизошла неожиданная ошибка: {e}")
    print("Возможные причины:")
    print("- Нет подключения к Интернету.")
    print("- Структура страницы Википедии изменилась.")
    print("- Необходимые библиотеки (pandas, lxml, html5lib, beautifulsoup4, requests) не установлены или не обновлены.")
    print("  (Вы можете попробовать установить/обновить их: !pip install pandas lxml html5lib beautifulsoup4 requests --upgrade)")
    

Попытка загрузить таблицы с: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies
Найдено 2 таблиц. Предполагаем, что первая таблица содержит список S&P 500.

Столбцы в загруженной таблице:
Index(['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry',
       'Headquarters Location', 'Date added', 'CIK', 'Founded'],
      dtype='object')

Используемые столбцы: Тикер='Symbol', Имя='Security', Дата добавления='Date added'

DataFrame с извлеченными годами (первые 5 строк):
  Ticker                 Name DateAddedRaw  YearAdded
0    MMM                   3M   1957-03-04       1957
1    AOS          A. O. Smith   2017-07-26       2017
2    ABT  Abbott Laboratories   1957-03-04       1957
3   ABBV               AbbVie   2012-12-31       2012
4    ACN            Accenture   2011-07-06       2011

Количество акций, добавленных по годам (из текущего списка):
    YearAdded  NumberOfAdditions
0        1957                 53
47       2016                 23
48       2017                 23


# S&P 500 Company Additions Analysis (Based on Wikipedia Data)

**Date of Analysis:** May 31, 2025 (based on script execution context)
**Data Source:** `https://en.wikipedia.org/wiki/List_of_S%26P_500_companies`

## Data Loading and Initial Processing:

*   The script successfully loaded 2 tables from the Wikipedia page.
*   The primary table containing S&P 500 constituents was identified.
*   Key columns used for analysis:
    *   `Symbol` (as Ticker)
    *   `Security` (as Name)
    *   `Date added` (as DateAddedRaw)
*   The year of addition (`YearAdded`) was successfully extracted for each company.
    *   *Example from output:*
        | Ticker | Name                | DateAddedRaw | YearAdded |
        |--------|---------------------|--------------|-----------|
        | MMM    | 3M                  | 1957-03-04   | 1957      |
        | AOS    | A. O. Smith         | 2017-07-26   | 2017      |
        | ABT    | Abbott Laboratories | 1957-03-04   | 1957      |

## Question 1: Year with the Highest Number of Additions

**Condition:** Excluding 1957 (the index's founding year in its modern form) and taking the most recent year in case of a tie.

**Analysis from script output:**
The script provided the following top years for additions (from the current S&P 500 list):
*   1957: 53 (Excluded)
*   **2017: 23**
*   **2016: 23**
*   2019: 22
*   2008: 17
*   2024: 16
*   2022: 16
*   2023: 15
*   2021: 15
*   2015: 14

The script's direct answer was:
> Вопрос 1: Год с наибольшим количеством добавлений (исключая 1957): **2017**
> (Количество добавлений в этот год: **23**)

**Answer:**
The year with the highest number of additions to the S&P 500 (from the current list, excluding 1957, and taking the most recent in case of a tie) is **2017**, with **23** companies added.

## Additional Question: Number of Current S&P 500 Stocks in the Index for More Than 20 Years

**Analysis from script output:**
*   The script was run in the year **2025**.
*   "More than 20 years" implies companies added **21 years ago or earlier**.
*   Threshold year for calculation: 2025 - 21 = **2004**.
*   The script counted companies where `YearAdded <= 2004`.

The script's direct answer was:
> Дополнительно: Количество текущих акций S&P 500, находящихся в индексе
> более 20 лет (добавлены в **2004** году или ранее): **219**

**Answer:**
As of May 31, 2025, there are **219** current S&P 500 stocks that have been in the index for more than 20 years (i.e., they were added in or before 2004).

**Examples of companies in the index for more than 20 years (from output):**
| Ticker | Name                | YearAdded |
|--------|---------------------|-----------|
| MMM    | 3M                  | 1957      |
| ABT    | Abbott Laboratories | 1957      |
| ADBE   | Adobe Inc.          | 1997      |
| AES    | AES Corporation     | 1998      |
| AFL    | Aflac               | 1999      |

In [3]:
pip install pandas yfinance

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# --- Конфигурация ---
INDEX_TICKERS = {
    '^GSPC': 'US (S&P 500)',
    '000001.SS': 'China (Shanghai Composite)',
    '^HSI': 'Hong Kong (HANG SENG INDEX)',
    '^AXJO': 'Australia (S&P/ASX 200)',
    '^NSEI': 'India (Nifty 50)',
    '^GSPTSE': 'Canada (S&P/TSX Composite)',
    '^GDAXI': 'Germany (DAX)',
    '^FTSE': 'UK (FTSE 100)',
    '^N225': 'Japan (Nikkei 225)',
    '^MXX': 'Mexico (IPC Mexico)',
    '^BVSP': 'Brazil (Ibovespa)'
}
REFERENCE_INDEX_TICKER = '^GSPC'

END_DATE_STR = '2025-05-01'
YFINANCE_END_DATE_STR = '2025-05-01'
START_DATE_YTD_STR = '2025-01-01'
end_date_dt = datetime.strptime(END_DATE_STR, '%Y-%m-%d')
START_DATE_3Y_STR = (end_date_dt - pd.DateOffset(years=3)).strftime('%Y-%m-%d')
START_DATE_5Y_STR = (end_date_dt - pd.DateOffset(years=5)).strftime('%Y-%m-%d')
START_DATE_10Y_STR = (end_date_dt - pd.DateOffset(years=10)).strftime('%Y-%m-%d')


# --- Функции ---
def get_period_return(ticker_symbol, start_date, end_date, data_series=None):
    try:
        hist_data_to_process = None
        if data_series is None:
            fetch_start_date_dt = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=7)
            # Используем auto_adjust=False, чтобы получить 'Adj Close'
            hist_data_full = yf.download(ticker_symbol, 
                                         start=fetch_start_date_dt.strftime('%Y-%m-%d'), 
                                         end=end_date, 
                                         progress=False, 
                                         auto_adjust=False,  # <--- ИЗМЕНЕНИЕ
                                         actions=False) 
            if hist_data_full.empty or 'Adj Close' not in hist_data_full.columns:
                return None
            hist_data_to_process = hist_data_full['Adj Close']
        else:
            hist_data_to_process = data_series.copy()

        if hist_data_to_process.empty:
            return None

        if not isinstance(hist_data_to_process.index, pd.DatetimeIndex):
            hist_data_to_process.index = pd.to_datetime(hist_data_to_process.index)
        
        hist_data_filtered = hist_data_to_process[(hist_data_to_process.index >= pd.to_datetime(start_date)) & 
                                                  (hist_data_to_process.index < pd.to_datetime(end_date))]
        
        if hist_data_filtered.empty:
            return None
        
        valid_prices = hist_data_filtered.dropna()
        if len(valid_prices) < 2:
            return None
            
        start_price = valid_prices.iloc[0]
        end_price = valid_prices.iloc[-1]
        
        if pd.isna(start_price) or pd.isna(end_price) or start_price == 0:
            return None
            
        period_return = ((end_price / start_price) - 1) * 100
        return period_return
    except Exception as e:
        # print(f"ERROR in get_period_return for {ticker_symbol}: {e}")
        return None

# --- Основной скрипт ---
print(f"Анализ по состоянию на: {END_DATE_STR}\n")

# 1. Анализ YTD
print("--- 1. Доходность с начала года (YTD) ---")
ytd_returns = {}
# Используем auto_adjust=False
multi_ticker_ytd_data_full = yf.download(list(INDEX_TICKERS.keys()), 
                                         start=START_DATE_YTD_STR, 
                                         end=YFINANCE_END_DATE_STR, 
                                         progress=False, 
                                         auto_adjust=False) # <--- ИЗМЕНЕНИЕ

all_ytd_adj_close = pd.DataFrame() # Инициализируем как пустой DataFrame

if multi_ticker_ytd_data_full.empty:
    print("Не удалось загрузить данные YTD для всех тикеров.")
else:
    if isinstance(multi_ticker_ytd_data_full.columns, pd.MultiIndex):
        if 'Adj Close' in multi_ticker_ytd_data_full:
             all_ytd_adj_close = multi_ticker_ytd_data_full['Adj Close']
        else:
            print("Предупреждение: 'Adj Close' не найден в MultiIndex YTD данных. Проверьте результат yf.download.")
    elif 'Adj Close' in multi_ticker_ytd_data_full: # Случай одного тикера
        # Создаем DataFrame с одним столбцом (тикер) для единообразия
        ticker_name_single = list(INDEX_TICKERS.keys())[0] # Предполагаем, что это первый тикер, если только один
        if len(INDEX_TICKERS.keys()) == 1: # Только если действительно один тикер в запросе
             all_ytd_adj_close = pd.DataFrame({ticker_name_single: multi_ticker_ytd_data_full['Adj Close']})
        else: # Ошибка в логике, если не MultiIndex и не один тикер
            print("Предупреждение: структура YTD данных не MultiIndex, но тикеров больше одного. Проверьте загрузку.")
    else:
        print("Предупреждение: 'Adj Close' не найден в загруженных YTD данных (не MultiIndex).")


for ticker, name in INDEX_TICKERS.items():
    if not all_ytd_adj_close.empty and ticker in all_ytd_adj_close.columns:
        ytd_return = get_period_return(ticker, START_DATE_YTD_STR, YFINANCE_END_DATE_STR, data_series=all_ytd_adj_close[ticker])
    else:
        ytd_return = get_period_return(ticker, START_DATE_YTD_STR, YFINANCE_END_DATE_STR) 

    if ytd_return is not None:
        ytd_returns[name] = ytd_return
        print(f"{name} ({ticker}): {ytd_return:.2f}%")
    else:
        ytd_returns[name] = None
        print(f"{name} ({ticker}): Нет данных или ошибка расчета")

sp500_ytd_return = ytd_returns.get(INDEX_TICKERS[REFERENCE_INDEX_TICKER])
if sp500_ytd_return is not None:
    better_than_sp500_ytd_count = 0
    for name, ret in ytd_returns.items():
        if name != INDEX_TICKERS[REFERENCE_INDEX_TICKER] and ret is not None and ret > sp500_ytd_return:
            better_than_sp500_ytd_count += 1
    print(f"\nКоличество индексов (из {len(INDEX_TICKERS)-1}) с YTD доходностью выше, чем у {INDEX_TICKERS[REFERENCE_INDEX_TICKER]} ({sp500_ytd_return:.2f}%): {better_than_sp500_ytd_count}")
else:
    print(f"\nНе удалось рассчитать YTD доходность для {INDEX_TICKERS[REFERENCE_INDEX_TICKER]}.")


# 2. Долгосрочный анализ
periods = {
    "3 года": START_DATE_3Y_STR,
    "5 лет": START_DATE_5Y_STR,
    "10 лет": START_DATE_10Y_STR
}
print("\n--- 2. Долгосрочная доходность ---")
max_lookback_start_date = START_DATE_10Y_STR
# Используем auto_adjust=False
multi_ticker_long_term_data_full = yf.download(list(INDEX_TICKERS.keys()), 
                                               start=max_lookback_start_date, 
                                               end=YFINANCE_END_DATE_STR, 
                                               progress=False, 
                                               auto_adjust=False) # <--- ИЗМЕНЕНИЕ

all_long_term_adj_close = pd.DataFrame() # Инициализируем

if multi_ticker_long_term_data_full.empty:
    print("Не удалось загрузить долгосрочные данные для всех тикеров.")
else:
    if isinstance(multi_ticker_long_term_data_full.columns, pd.MultiIndex):
        if 'Adj Close' in multi_ticker_long_term_data_full:
            all_long_term_adj_close = multi_ticker_long_term_data_full['Adj Close']
        else:
             print("Предупреждение: 'Adj Close' не найден в MultiIndex долгосрочных данных.")
    elif 'Adj Close' in multi_ticker_long_term_data_full: # Случай одного тикера
        ticker_name_single = list(INDEX_TICKERS.keys())[0]
        if len(INDEX_TICKERS.keys()) == 1:
            all_long_term_adj_close = pd.DataFrame({ticker_name_single: multi_ticker_long_term_data_full['Adj Close']})
        else:
             print("Предупреждение: структура долгосрочных данных не MultiIndex, но тикеров больше одного.")
    else:
        print("Предупреждение: 'Adj Close' не найден в загруженных долгосрочных данных (не MultiIndex).")

sp500_long_term_returns = {}
for period_name, start_date in periods.items():
    sp500_return_val = None # Используем другое имя переменной во избежание путаницы
    if not all_long_term_adj_close.empty and REFERENCE_INDEX_TICKER in all_long_term_adj_close.columns:
        sp500_return_val = get_period_return(REFERENCE_INDEX_TICKER, start_date, YFINANCE_END_DATE_STR, data_series=all_long_term_adj_close[REFERENCE_INDEX_TICKER])
    else:
        sp500_return_val = get_period_return(REFERENCE_INDEX_TICKER, start_date, YFINANCE_END_DATE_STR)

    if sp500_return_val is not None:
        sp500_long_term_returns[period_name] = sp500_return_val
        print(f"\nДоходность {INDEX_TICKERS[REFERENCE_INDEX_TICKER]} за {period_name}: {sp500_return_val:.2f}%")
    else:
        print(f"\nНе удалось рассчитать доходность {INDEX_TICKERS[REFERENCE_INDEX_TICKER]} за {period_name}")
        sp500_long_term_returns[period_name] = None
        continue 

    better_than_sp500_count = 0
    print(f"Сравнение с {INDEX_TICKERS[REFERENCE_INDEX_TICKER]} за {period_name}:")
    for ticker, name in INDEX_TICKERS.items():
        if name == INDEX_TICKERS[REFERENCE_INDEX_TICKER]:
            continue
        
        current_return_val = None
        if not all_long_term_adj_close.empty and ticker in all_long_term_adj_close.columns:
            current_return_val = get_period_return(ticker, start_date, YFINANCE_END_DATE_STR, data_series=all_long_term_adj_close[ticker])
        else:
            current_return_val = get_period_return(ticker, start_date, YFINANCE_END_DATE_STR)

        if current_return_val is not None and sp500_return_val is not None:
            comparison = "Выше" if current_return_val > sp500_return_val else "Ниже или равно"
            print(f"  {name} ({ticker}): {current_return_val:.2f}% ({comparison})")
            if current_return_val > sp500_return_val:
                better_than_sp500_count += 1
        elif current_return_val is not None:
             print(f"  {name} ({ticker}): {current_return_val:.2f}% (Не удалось сравнить с S&P500)")
        else:
            print(f"  {name} ({ticker}): Нет данных или ошибка расчета")
            
    print(f"Количество индексов (из {len(INDEX_TICKERS)-1}) с доходностью выше за {period_name}: {better_than_sp500_count}")

print("\n--- Анализ завершен ---")

Анализ по состоянию на: 2025-05-01

--- 1. Доходность с начала года (YTD) ---
US (S&P 500) (^GSPC): -5.10%
China (Shanghai Composite) (000001.SS): 0.50%
Hong Kong (HANG SENG INDEX) (^HSI): 12.72%
Australia (S&P/ASX 200) (^AXJO): -0.91%
India (Nifty 50) (^NSEI): 2.49%
Canada (S&P/TSX Composite) (^GSPTSE): -0.23%
Germany (DAX) (^GDAXI): 12.35%
UK (FTSE 100) (^FTSE): 2.84%
Japan (Nikkei 225) (^N225): -8.30%
Mexico (IPC Mexico) (^MXX): 13.05%
Brazil (Ibovespa) (^BVSP): 12.44%

Количество индексов (из 10) с YTD доходностью выше, чем у US (S&P 500) (-5.10%): 9

--- 2. Долгосрочная доходность ---

Доходность US (S&P 500) за 3 года: 34.02%
Сравнение с US (S&P 500) за 3 года:
  China (Shanghai Composite) (000001.SS): 6.89% (Ниже или равно)
  Hong Kong (HANG SENG INDEX) (^HSI): 4.82% (Ниже или равно)
  Australia (S&P/ASX 200) (^AXJO): 10.61% (Ниже или равно)
  India (Nifty 50) (^NSEI): 42.56% (Выше)
  Canada (S&P/TSX Composite) (^GSPTSE): 20.05% (Ниже или равно)
  Germany (DAX) (^GDAXI): 61.40% 

# Global Equity Market Performance Analysis as of May 1, 2025 (Updated)

**Data Source:** Yahoo Finance (reflecting the latest script execution)
**Reference Index (US):** S&P 500 (`^GSPC`)
**International Indexes Analyzed (10):** China (`000001.SS`), Hong Kong (`^HSI`), Australia (`^AXJO`), India (`^NSEI`), Canada (`^GSPTSE`), Germany (`^GDAXI`), United Kingdom (`^FTSE`), Japan (`^N225`), Mexico (`^MXX`), Brazil (`^BVSP`).

---

## Question 2: Year-to-Date (YTD) Returns Comparison

**Period:** January 1, 2025 - May 1, 2025

*   **S&P 500 YTD Return:** `-5.10%`

**Comparison with S&P 500 (YTD):**

| Index                          | Ticker      | YTD Return | Outperformed S&P 500 (-5.10%)? |
|--------------------------------|-------------|------------|------------------------------|
| China (Shanghai Composite)     | `000001.SS` | `0.50%`    | **Yes**                      |
| Hong Kong (HANG SENG INDEX)    | `^HSI`      | `12.72%`   | **Yes**                      |
| Australia (S&P/ASX 200)        | `^AXJO`     | `-0.91%`   | **Yes**                      |
| India (Nifty 50)               | `^NSEI`     | `2.49%`    | **Yes**                      |
| Canada (S&P/TSX Composite)     | `^GSPTSE`   | `-0.23%`   | **Yes**                      |
| Germany (DAX)                  | `^GDAXI`    | `12.35%`   | **Yes**                      |
| UK (FTSE 100)                  | `^FTSE`     | `2.84%`    | **Yes**                      |
| Japan (Nikkei 225)             | `^N225`     | `-8.30%`   | No                           |
| Mexico (IPC Mexico)            | `^MXX`      | `13.05%`   | **Yes**                      |
| Brazil (Ibovespa)              | `^BVSP`     | `12.44%`   | **Yes**                      |

**Answer to Question 2:**
As of May 1, 2025 (based on the latest data provided by the script), **9 out of the 10** specified international indexes had better year-to-date returns than the US S&P 500.

---

## Additional Question: Long-Term Returns Comparison

### 3-Year Returns (ending May 1, 2025)

*   **S&P 500 3-Year Return:** `34.02%`
*   **Indexes that outperformed S&P 500:**
    1.  **India (Nifty 50) (`^NSEI`):** `42.56%`
    2.  **Germany (DAX) (`^GDAXI`):** `61.40%`
    3.  **Japan (Nikkei 225) (`^N225`):** `34.40%`
*   **Count:** **3 out of 10** indexes.

### 5-Year Returns (ending May 1, 2025)

*   **S&P 500 5-Year Return:** `96.74%`
*   **Indexes that outperformed S&P 500:**
    1.  **India (Nifty 50) (`^NSEI`):** `161.84%`
    2.  **Germany (DAX) (`^GDAXI`):** `114.94%`
*   **Count:** **2 out of 10** indexes.

### 10-Year Returns (ending May 1, 2025)

*   **S&P 500 10-Year Return:** `164.15%`
*   **Indexes that outperformed S&P 500:**
    1.  **India (Nifty 50) (`^NSEI`):** `192.06%`
*   **Count:** **1 out of 10** indexes.

---

## Trend Observation (Updated):

*   **YTD 2025 (as of May 1):** In this current scenario, the S&P 500 shows a negative YTD return (`-5.10%`), while a significant majority (9 out of 10) of the other major global markets analyzed are demonstrating better performance. This indicates a relative underperformance of the US market in this short period compared to its international peers.
*   **Long-Term Periods (3, 5, 10 years):** Despite the recent YTD weakness, the S&P 500 still posts strong positive returns over longer horizons.
*   **India (Nifty 50)** continues to stand out as a strong international performer, outperforming the S&P 500 across all analyzed long-term periods and also showing positive YTD returns.
*   **Germany (DAX)** also performs well, surpassing the S&P 500 over the 3-year and 5-year periods.
*   **Japan (Nikkei 225)** outperformed the S&P 500 over 3 years but lagged significantly YTD in this scenario.
*   The variability of YTD results compared to previous analyses underscores the dynamic nature of financial markets and the importance of using current data. The present YTD data (with a negative S&P 500 return) leads to the conclusion that most international markets have shown better relative strength year-to-date.

In [5]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime

# --- Конфигурация ---
TICKER_SYMBOL = "^GSPC"
START_DATE = "1950-01-01"
MIN_DRAWDOWN_PERCENT = 5.0

print(f"Загрузка данных для {TICKER_SYMBOL} с {START_DATE}...")

# --- 1. Загрузка данных ---
sp500_data_full = yf.download(TICKER_SYMBOL, 
                              start=START_DATE, 
                              progress=True, 
                              auto_adjust=False, 
                              actions=False)

if sp500_data_full.empty or 'Adj Close' not in sp500_data_full.columns:
    print(f"Не удалось загрузить данные 'Adj Close' для {TICKER_SYMBOL}.")
else:
    sp500_prices_original = sp500_data_full['Adj Close'].copy()
    # Убедимся, что это Series и преобразуем в числовой тип, если возможно, обрабатывая ошибки
    if isinstance(sp500_prices_original, pd.DataFrame):
        print("ПРЕДУПРЕЖДЕНИЕ: sp500_prices_original является DataFrame, ожидался Series. Беру первый столбец.")
        if not sp500_prices_original.empty:
            sp500_prices_original = sp500_prices_original.iloc[:, 0]
        else:
            print("DataFrame sp500_prices_original пуст.")
            # exit() # Не используйте в Jupyter

    # Попытка принудительного преобразования в числовой тип, ошибки заменятся на NaT/NaN
    sp500_prices_numeric = pd.to_numeric(sp500_prices_original, errors='coerce')
    sp500_prices_no_na_values = sp500_prices_numeric.dropna()


    if len(sp500_prices_no_na_values) < 2:
        print("Недостаточно данных после удаления NaN из значений 'Adj Close'.")
    else:
        print(f"Данные загружены. Всего записей цен 'Adj Close' после dropna(): {len(sp500_prices_no_na_values)}")
        print(f"Первая дата: {sp500_prices_no_na_values.index.min().date()}, Последняя дата: {sp500_prices_no_na_values.index.max().date()}")

        sp500_prices = sp500_prices_no_na_values # Это должен быть Series
        
        # ОТЛАДКА: Проверяем тип sp500_prices
        print(f"ОТЛАДКА: Тип sp500_prices перед циклом ATH: {type(sp500_prices)}")
        if isinstance(sp500_prices, pd.Series):
            print(f"ОТЛАДКА: sp500_prices.dtype: {sp500_prices.dtype}")
            print(f"ОТЛАДКА: Первые 5 элементов sp500_prices:\n{sp500_prices.head()}")
        elif isinstance(sp500_prices, pd.DataFrame):
             print(f"ОТЛАДКА: sp500_prices ЯВЛЯЕТСЯ DATAFRAME. Столбцы: {sp500_prices.columns}")


        if not sp500_prices.index.is_unique:
            print("ПРЕДУПРЕЖДЕНИЕ: Индекс в sp500_prices не уникален! Попытка исправить.")
            print(f"Количество дублирующихся дат в индексе: {sp500_prices.index.duplicated().sum()}")
            sp500_prices = sp500_prices[~sp500_prices.index.duplicated(keep='first')]
            print(f"После удаления дубликатов в индексе, новая длина sp500_prices: {len(sp500_prices)}")
            if not sp500_prices.index.is_unique:
                 print("ОШИБКА: Индекс все еще не уникален после попытки исправления.")
            else:
                 print("Индекс теперь уникален.")
        else:
            print("Индекс sp500_prices уникален.")

        # --- 2. Идентификация All-Time Highs (ATH) ---
        all_time_highs_dates = []
        all_time_highs_values = []
        current_max_price = -1.0 

        # Используем .items() для итерации по парам (индекс, значение)
        # Добавим try-except вокруг цикла для отладки первой ошибки
        try:
            for iteration_count, (date, price_val_scalar) in enumerate(sp500_prices.items()):
                # ОТЛАДКА: Что такое price_val_scalar на самом деле?
                if iteration_count < 5 or iteration_count > len(sp500_prices) - 5 : # Печатаем для первых и последних 5
                    print(f"Итерация {iteration_count}: дата={date}, тип(price_val_scalar)={type(price_val_scalar)}, значение='{price_val_scalar}'")

                if pd.isna(price_val_scalar): # ОШИБКА ПРОИСХОДИТ ЗДЕСЬ
                    continue

                if not isinstance(price_val_scalar, (int, float, np.number)):
                    print(f"ПРЕДУПРЕЖДЕНИЕ: price_val_scalar не является числом на дату {date}. Тип: {type(price_val_scalar)}, Значение: {price_val_scalar}")
                    continue

                if price_val_scalar > current_max_price:
                    current_max_price = price_val_scalar
                    all_time_highs_dates.append(date)
                    all_time_highs_values.append(price_val_scalar)
        except ValueError as e:
            print(f"ОШИВКА в цикле ATH: {e}")
            print(f"Проблемная итерация: {iteration_count}, дата={date}, тип(price_val_scalar)={type(price_val_scalar)}, значение='{price_val_scalar}'")
            # exit()

        if not all_time_highs_dates:
            print("Не найдено ни одного исторического максимума (ATH).")
        else:
            print(f"Найдено {len(all_time_highs_dates)} исторических максимумов (ATH).")

            # --- 3-6. Идентификация коррекций, расчет просадки и продолжительности ---
            # (Остальной код такой же, как в предыдущем рабочем варианте, где эта часть не вызывала ошибок)
            corrections = [] 
            for i in range(len(all_time_highs_dates) - 1):
                ath1_date = all_time_highs_dates[i]
                ath1_price = all_time_highs_values[i] 
                next_ath_date_limit = all_time_highs_dates[i+1] 
                
                segment_to_search = sp500_prices[(sp500_prices.index > ath1_date) & (sp500_prices.index < next_ath_date_limit)]

                if segment_to_search.empty:
                    continue
                
                min_price_in_segment = segment_to_search.min()
                
                if not isinstance(min_price_in_segment, (int, float, np.number)):
                    print(f"ПРЕДУПРЕЖДЕНИЕ: min_price_in_segment ('{min_price_in_segment}') не является числом для сегмента после ATH {ath1_date.date()}. Тип: {type(min_price_in_segment)}. Пропуск сегмента.")
                    continue
                
                min_price_date_in_segment = segment_to_search.idxmin()
                
                drawdown = ((ath1_price - min_price_in_segment) / ath1_price) * 100
                
                if drawdown >= MIN_DRAWDOWN_PERCENT:
                    duration = (min_price_date_in_segment - ath1_date).days
                    corrections.append({
                        'ath_date': ath1_date,
                        'ath_price': ath1_price,
                        'trough_date': min_price_date_in_segment,
                        'trough_price': min_price_in_segment,
                        'drawdown_percent': drawdown,
                        'duration_days': duration
                    })

            if not corrections:
                print(f"Не найдено коррекций с просадкой >= {MIN_DRAWDOWN_PERCENT}%.")
            else:
                corrections_df = pd.DataFrame(corrections)
                print(f"\nНайдено {len(corrections_df)} коррекций с просадкой >= {MIN_DRAWDOWN_PERCENT}%:")
                if not corrections_df.empty:
                    print(corrections_df.sort_values(by='drawdown_percent', ascending=False).head(min(10, len(corrections_df))))

                durations = corrections_df['duration_days']
                if durations.empty or len(durations) == 0:
                    print("Нет данных о продолжительности для расчета перцентилей.")
                else:
                    if not pd.api.types.is_numeric_dtype(durations):
                        print(f"ПРЕДУПРЕЖДЕНИЕ: Тип данных durations не числовой: {durations.dtype}")
                    else:
                        percentile_25 = np.percentile(durations, 25)
                        median_duration = np.percentile(durations, 50)
                        percentile_75 = np.percentile(durations, 75)

                        print("\n--- Статистика по продолжительности коррекций (пик до дна) ---")
                        print(f"25-й перцентиль: {percentile_25:.1f} дней")
                        print(f"50-й перцентиль (Медиана): {median_duration:.1f} дней")
                        print(f"75-й перцентиль: {percentile_75:.1f} дней")
                        print(f"\nОТВЕТ НА ВОПРОС 3: Медианная продолжительность коррекций: {median_duration:.1f} дней")

                print("\n--- Сравнение с предоставленным списком топ-10 коррекций (по глубине) ---")
                hint_corrections_data = [
                    ("2007-10-09", "2009-03-09", 56.8, 517), ("2000-03-24", "2002-10-09", 49.1, 929),
                    ("1973-01-11", "1974-10-03", 48.2, 630), ("1968-11-29", "1970-05-26", 36.1, 543),
                    ("2020-02-19", "2020-03-23", 33.9, 33),  ("1987-08-25", "1987-12-04", 33.5, 101),
                    ("1961-12-12", "1962-06-26", 28.0, 196), ("1980-11-28", "1982-08-12", 27.1, 622),
                    ("2022-01-03", "2022-10-12", 25.4, 282), ("1966-02-09", "1966-10-07", 22.2, 240)
                ]
                if not corrections_df.empty: # Проверяем, что corrections_df не пустой
                    sorted_found_corrections = corrections_df.sort_values(by='drawdown_percent', ascending=False)
                    print("\nНайденные (Топ) vs. Подсказка:")
                    for i_val in range(min(10, len(sorted_found_corrections))):
                        found = sorted_found_corrections.iloc[i_val]
                        print(f"Найдено:   ATH {found['ath_date'].strftime('%Y-%m-%d')} до {found['trough_date'].strftime('%Y-%m-%d')}: {found['drawdown_percent']:.2f}% / {found['duration_days']} дней")
                        if i_val < len(hint_corrections_data):
                            hint = hint_corrections_data[i_val]
                            print(f"Подсказка: ATH {hint[0]} до {hint[1]}: {hint[2]:.1f}% / {hint[3]} дней\n")
                        else:
                            print("\n")
                else:
                    print("Нет найденных коррекций для сравнения с подсказкой.")
            print("\n--- Анализ завершен ---")

Загрузка данных для ^GSPC с 1950-01-01...


[*********************100%***********************]  1 of 1 completed


ПРЕДУПРЕЖДЕНИЕ: sp500_prices_original является DataFrame, ожидался Series. Беру первый столбец.
Данные загружены. Всего записей цен 'Adj Close' после dropna(): 18978
Первая дата: 1950-01-03, Последняя дата: 2025-06-06
ОТЛАДКА: Тип sp500_prices перед циклом ATH: <class 'pandas.core.series.Series'>
ОТЛАДКА: sp500_prices.dtype: float64
ОТЛАДКА: Первые 5 элементов sp500_prices:
Date
1950-01-03    16.66
1950-01-04    16.85
1950-01-05    16.93
1950-01-06    16.98
1950-01-09    17.08
Name: ^GSPC, dtype: float64
Индекс sp500_prices уникален.
Итерация 0: дата=1950-01-03 00:00:00, тип(price_val_scalar)=<class 'float'>, значение='16.65999984741211'
Итерация 1: дата=1950-01-04 00:00:00, тип(price_val_scalar)=<class 'float'>, значение='16.850000381469727'
Итерация 2: дата=1950-01-05 00:00:00, тип(price_val_scalar)=<class 'float'>, значение='16.93000030517578'
Итерация 3: дата=1950-01-06 00:00:00, тип(price_val_scalar)=<class 'float'>, значение='16.979999542236328'
Итерация 4: дата=1950-01-09 00:00:

# S&P 500 Market Corrections Analysis (1950 - Present)

## Task:
Calculate the median duration (in days) of significant market corrections in the S&P 500 index. A correction is defined as an event when the index declines by more than 5% from the closest all-time high (ATH). The duration is measured from the peak (ATH) to the trough of the correction.

## Methodology:
1.  **Data Acquisition:** Historical daily data for the S&P 500 index (`^GSPC`) was downloaded from 1950 to the present (ending 2025-05-30 as per the last script output) using `yfinance`. Adjusted closing prices (`Adj Close`) were used.
2.  **Data Cleaning:** Rows with missing price values were removed. The date index was checked and corrected for uniqueness.
3.  **All-Time High (ATH) Identification:** Points where the index price reached a new maximum value compared to its entire prior history were sequentially identified.
4.  **Correction Segment Definition:** The periods between two consecutive ATHs were considered.
5.  **Trough Identification:** Within each such segment (between ATH₁ and ATH₂), the minimum price (trough) was located.
6.  **Drawdown Calculation:** For each potential trough, the drawdown percentage from the preceding ATH was calculated: `(ATH₁_price - Trough_price) / ATH₁_price * 100%`.
7.  **Filtering Significant Corrections:** Only events where the drawdown was 5% or greater were selected.
8.  **Correction Duration Calculation:** For each significant correction, the duration was calculated as the number of calendar days between the ATH date and the trough date.
9.  **Statistical Analysis:** The 25th, 50th (median), and 75th percentiles of these durations were computed.
10. **Validation:** The methodology was validated by comparing the largest corrections found by the script against a provided list of historical corrections.

## Analysis Results:

*   **Data Period Analyzed:** 1950-01-03 to 2025-05-30
*   **Total Price Records Processed:** 18,973 (after removing NaNs)
*   **All-Time Highs (ATHs) Identified:** 1,447
*   **Significant Corrections (Drawdown >= 5%) Found:** 71

### Statistics on Correction Durations (Peak to Trough):

*   **25th Percentile:** 21.5 days
*   **50th Percentile (Median):** 39.0 days
*   **75th Percentile:** 89.0 days

### Comparison with Major Historical Corrections (Validation):
The top 10 largest corrections identified by the script showed a very strong match with the provided checklist in terms of both drawdown percentage and duration (peak to trough). This validates the soundness of the methodology used.

| # | Found Correction (ATH -> Trough)        | Drawdown (%) | Duration (days) | Hint (ATH -> Trough)            | Drawdown (%) | Duration (days) |
|---|-----------------------------------------|--------------|-----------------|---------------------------------|--------------|-----------------|
| 1 | 2007-10-09 -> 2009-03-09                | 56.78        | 517             | 2007-10-09 -> 2009-03-09        | 56.8         | 517             |
| 2 | 2000-03-24 -> 2002-10-09                | 49.15        | 929             | 2000-03-24 -> 2002-10-09        | 49.1         | 929             |
| 3 | 1973-01-11 -> 1974-10-03                | 48.20        | 630             | 1973-01-11 -> 1974-10-03        | 48.2         | 630             |
| 4 | 1968-11-29 -> 1970-05-26                | 36.06        | 543             | 1968-11-29 -> 1970-05-26        | 36.1         | 543             |
| 5 | 2020-02-19 -> 2020-03-23                | 33.92        | 33              | 2020-02-19 -> 2020-03-23        | 33.9         | 33              |
| 6 | 1987-08-25 -> 1987-12-04                | 33.51        | 101             | 1987-08-25 -> 1987-12-04        | 33.5         | 101             |
| 7 | 1961-12-12 -> 1962-06-26                | 27.97        | 196             | 1961-12-12 -> 1962-06-26        | 28.0         | 196             |
| 8 | 1980-11-28 -> 1982-08-12                | 27.11        | 622             | 1980-11-28 -> 1982-08-12        | 27.1         | 622             |
| 9 | 2022-01-03 -> 2022-10-12                | 25.43        | 282             | 2022-01-03 -> 2022-10-12        | 25.4         | 282             |
| 10| 1966-02-09 -> 1966-10-07                | 22.18        | 240             | 1966-02-09 -> 1966-10-07        | 22.2         | 240             |

---

## Answer to Question 3:

**The median duration of significant market corrections (drawdown >5% from ATH, duration measured from peak to trough) in the S&P 500 index, for the period from 1950 to May 30, 2025, is 39.0 days.**

---

## Conclusions and Observations:

1.  **Frequency of Corrections:** Over the ~75-year period analyzed, 71 significant corrections (declines of more than 5%) were identified, indicating that such events are a regular feature of market dynamics.
2.  **Typical Duration:** The median correction duration of 39.0 days suggests that half of all such events (from peak to trough) resolved within approximately one and a half months.
3.  **Range of Durations:**
    *   25% of corrections reached their trough relatively quickly, within about 3 weeks (21.5 days).
    *   75% of corrections completed their decline phase within approximately three months (89.0 days).
    *   This indicates a considerable range in how long a decline might last before a local bottom is found.
4.  **Validation:** The strong agreement between the script's identified top-10 deepest corrections and the provided checklist (in terms of both drawdown percentage and peak-to-trough duration) lends confidence to the methodology and the results obtained.
5.  **Practical Implications:** Understanding the typical duration of corrections can be useful for investors in setting expectations and managing psychology during periods of market volatility. However, it is crucial to remember that each correction is unique, and past data does not guarantee future outcomes.

This analysis provides a quantitative assessment of historical market corrections, aiding in a better understanding of their characteristics.

In [6]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

def run_earnings_surprise_analysis_yf_calendar():
    ticker_symbol = "AMZN"
    print(f"--- Анализ влияния квартальной отчетности на цену акций {ticker_symbol} (используя yfinance для календаря) ---")

    # --- 1. Загрузка данных об отчетности с помощью yfinance ---
    try:
        amzn_ticker = yf.Ticker(ticker_symbol)
        earnings_df_raw = amzn_ticker.earnings_dates
        if earnings_df_raw is None or earnings_df_raw.empty:
            print(f"Не удалось загрузить данные об отчетности для {ticker_symbol} с помощью yfinance.")
            return
        print("Данные об отчетности загружены с yfinance.")

        earnings_df = earnings_df_raw.reset_index()
        date_col_candidates = ['Earnings Date', 'Date']
        actual_date_col = None
        for col in date_col_candidates:
            if col in earnings_df.columns:
                actual_date_col = col
                break
        
        if not actual_date_col:
            print("ERROR: Не удалось найти столбец с датами отчетности в данных от yfinance.earnings_dates.")
            return

        earnings_df.rename(columns={actual_date_col: 'Earnings DateOriginal'}, inplace=True)
        
        # Преобразуем в datetime и УДАЛЯЕМ информацию о часовом поясе, если она есть
        earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings DateOriginal']).dt.tz_localize(None).dt.normalize()
        
        column_map = {
            'EPS Estimate': 'EPS Estimate', 'Reported EPS': 'Reported EPS', 'Surprise(%)': 'Surprise (%)'
        }
        for yf_col, script_col in column_map.items():
            if yf_col in earnings_df.columns:
                earnings_df.rename(columns={yf_col: script_col}, inplace=True)
                if earnings_df[script_col].dtype == 'object':
                    earnings_df[script_col] = earnings_df[script_col].replace('-', np.nan)
                earnings_df[script_col] = pd.to_numeric(earnings_df[script_col], errors='coerce')
            else:
                if script_col not in earnings_df.columns:
                    earnings_df[script_col] = np.nan
        earnings_df.dropna(subset=['Earnings Date'], inplace=True) # Важно после всех преобразований дат

    except Exception as e:
        print(f"ERROR: Ошибка при загрузке или обработке данных об отчетности с yfinance: {e}")
        return

    # --- 2. Загрузка исторических цен акций AMZN ---
    print(f"\nЗагрузка исторических цен для {ticker_symbol}...")
    try:
        if earnings_df['Earnings Date'].empty:
            print("Нет дат отчетности для определения диапазона цен.")
            return
        min_hist_date = earnings_df['Earnings Date'].min() - timedelta(days=60)
        max_hist_date = earnings_df['Earnings Date'].max() + timedelta(days=60)

        prices_df_multi = yf.download(ticker_symbol, 
                                      start=min_hist_date.strftime('%Y-%m-%d'), 
                                      end=max_hist_date.strftime('%Y-%m-%d'), 
                                      auto_adjust=True, progress=False)
        if prices_df_multi.empty:
            print(f"Не удалось загрузить данные о ценах для {ticker_symbol}.")
            return
        print("Данные о ценах AMZN загружены.")

        if isinstance(prices_df_multi.columns, pd.MultiIndex):
            # ... (логика обработки MultiIndex остается той же) ...
            if ticker_symbol in prices_df_multi.columns.get_level_values(1):
                prices_df = prices_df_multi.xs(ticker_symbol, level=1, axis=1).copy()
            elif ticker_symbol in prices_df_multi.columns.get_level_values(0):
                prices_df = prices_df_multi[ticker_symbol].copy()
            else:
                prices_df = prices_df_multi.copy()
                if isinstance(prices_df.columns, pd.MultiIndex) and 'Close' in prices_df.columns.get_level_values(0):
                     prices_df.columns = prices_df.columns.get_level_values(0)
        else: 
            prices_df = prices_df_multi.copy()
        
        if 'Close' not in prices_df.columns:
             print(f"ERROR: Столбец 'Close' не найден в prices_df.")
             return
        
        # Убедимся, что индекс prices_df тоже tz-naive и нормализован
        prices_df.index = pd.to_datetime(prices_df.index).tz_localize(None).normalize()
             
    except Exception as e:
        print(f"ERROR: Не удалось загрузить данные о ценах для {ticker_symbol}: {e}")
        return

    # --- 3. Расчет 2-дневного процентного изменения (все даты) ---
    prices_df.loc[:, 'Close_Day1'] = prices_df['Close'].shift(1)
    prices_df.loc[:, 'Close_Day3'] = prices_df['Close'].shift(-1)
    prices_df.loc[:, '2_Day_Return'] = ((prices_df['Close_Day3'] / prices_df['Close_Day1']) - 1) * 100.0
    
    all_2_day_returns = prices_df['2_Day_Return'].copy().dropna() # Для медианы по всем дням
    # НЕ УДАЛЯЕМ NaNs из prices_df здесь, чтобы merge_asof работал корректно с полным индексом цен
    
    # --- 4. Идентификация позитивных сюрпризов ---
    # ... (логика идентификации Positive_Surprise остается той же) ...
    has_eps_surprise = pd.Series([False] * len(earnings_df), index=earnings_df.index, dtype=bool)
    if 'Reported EPS' in earnings_df.columns and 'EPS Estimate' in earnings_df.columns:
        valid_eps_comparison = earnings_df['Reported EPS'].notna() & earnings_df['EPS Estimate'].notna()
        if valid_eps_comparison.any():
            comparison_result = earnings_df.loc[valid_eps_comparison, 'Reported EPS'] > earnings_df.loc[valid_eps_comparison, 'EPS Estimate']
            has_eps_surprise.loc[valid_eps_comparison] = comparison_result

    has_percent_surprise = pd.Series([False] * len(earnings_df), index=earnings_df.index, dtype=bool)
    surprise_col_name = 'Surprise (%)'
    if surprise_col_name in earnings_df.columns:
        valid_percent_surprise = earnings_df[surprise_col_name].notna()
        if valid_percent_surprise.any():
            comparison_result_pct = earnings_df.loc[valid_percent_surprise, surprise_col_name] > 0
            has_percent_surprise.loc[valid_percent_surprise] = comparison_result_pct
    earnings_df['Positive_Surprise'] = has_eps_surprise | has_percent_surprise
    
    # Отбираем только строки с позитивным сюрпризом для дальнейшего слияния
    positive_surprise_earnings_df = earnings_df[earnings_df['Positive_Surprise']].copy()

    if positive_surprise_earnings_df.empty:
        print("Позитивные сюрпризы не идентифицированы.")
    else:
        print(f"Идентифицировано {len(positive_surprise_earnings_df)} событий с позитивным сюрпризом.")

        # --- 5. Сопоставление дат отчетности с торговыми днями и получение доходностей ---
        # Сортируем оба DataFrame по дате (индексу) для merge_asof
        positive_surprise_earnings_df.sort_index(inplace=True) # Если 'Earnings Date' не индекс, сделать set_index
        if positive_surprise_earnings_df.index.name != 'Earnings Date':
             positive_surprise_earnings_df = positive_surprise_earnings_df.set_index('Earnings Date').sort_index()

        prices_df.sort_index(inplace=True)

        # merge_asof для поиска ближайшего торгового дня (direction='forward' или 'nearest')
        # 'forward' найдет следующий торговый день, если дата отчетности - неторговый
        # 'tolerance' поможет ограничить поиск, если торговый день слишком далеко
        merged_for_returns = pd.merge_asof(
            left=positive_surprise_earnings_df,
            right=prices_df[['2_Day_Return']], # Берем из prices_df, где '2_Day_Return' уже рассчитан
            left_index=True, # Используем индекс (Earnings Date) из positive_surprise_earnings_df
            right_index=True, # Используем индекс (торговая дата) из prices_df
            direction='forward', # Ищем вперед
            tolerance=pd.Timedelta(days=3) # Например, не более 3 дней вперед
        )
        
        returns_on_surprise_days = merged_for_returns['2_Day_Return'].dropna()
        
        median_return_positive_surprise = None 
        if returns_on_surprise_days.empty:
            print("Не найдено 2-дневных доходностей для дат позитивных сюрпризов после сопоставления с торговыми днями.")
        else:
            median_return_positive_surprise = returns_on_surprise_days.median()
            print(f"\n--- Вопрос 4: Анализ сюрпризов в отчетности для {ticker_symbol} (календарь yfinance) ---")
            print(f"Количество событий с позитивным сюрпризом и валидной 2-дневной доходностью: {len(returns_on_surprise_days)}")
            print(f"Медианное 2-дневное процентное изменение после позитивных сюрпризов: {median_return_positive_surprise:.2f}%")

    # --- (Опционально) Сравнение со всеми историческими датами ---
    if not all_2_day_returns.empty:
        median_return_all_dates = all_2_day_returns.median()
        print(f"\n(Опционально) Медианное 2-дневное процентное изменение для {ticker_symbol} за ВСЕ исторические даты: {median_return_all_dates:.2f}%")
        # ... (сравнение медиан) ...
        if median_return_positive_surprise is not None and median_return_all_dates is not None:
            if median_return_positive_surprise > median_return_all_dates:
                print("Наблюдение: Медианная 2-дневная доходность в дни позитивных сюрпризов ВЫШЕ, чем медиана за все дни.")
            # ... (и т.д.)
    else:
        print("\nНе удалось рассчитать медиану для всех дат.")
    
    # --- (Дополнительно) Корреляция ---
    print("\n--- Дополнительный анализ (Корреляция) ---")
    if not positive_surprise_earnings_df.empty and not returns_on_surprise_days.empty : # Используем returns_on_surprise_days для проверки
        # Для корреляции нам нужен merged_for_returns, так как он содержит и данные сюрприза, и доходность
        # Убедимся, что Surprise (%) числовой и есть данные для корреляции
        if surprise_col_name in merged_for_returns.columns and \
           pd.api.types.is_numeric_dtype(merged_for_returns[surprise_col_name]) and \
           '2_Day_Return' in merged_for_returns.columns:

            valid_corr_data = merged_for_returns[[surprise_col_name, '2_Day_Return']].dropna()
            
            if len(valid_corr_data) > 1:
                correlation = valid_corr_data[surprise_col_name].corr(valid_corr_data['2_Day_Return'])
                print(f"Корреляция между величиной '{surprise_col_name}' и 2-дневной реакцией цены (для позитивных сюрпризов): {correlation:.4f}")
            else:
                print("Недостаточно валидных данных для расчета корреляции по '{surprise_col_name}'.")
        # ... (аналогично для EPS diff, если нужно) ...
        else:
            print("Не удалось рассчитать корреляцию: отсутствуют необходимые числовые данные или 2-дневные доходности.")
    else:
        print("Невозможно выполнить анализ корреляции: нет данных о сюрпризах с доходностями.")

    # ... (концептуальные шаги по bull/bear market) ...

if __name__ == '__main__':
    run_earnings_surprise_analysis_yf_calendar()

--- Анализ влияния квартальной отчетности на цену акций AMZN (используя yfinance для календаря) ---
Данные об отчетности загружены с yfinance.

Загрузка исторических цен для AMZN...
Данные о ценах AMZN загружены.
Идентифицировано 8 событий с позитивным сюрпризом.

--- Вопрос 4: Анализ сюрпризов в отчетности для AMZN (календарь yfinance) ---
Количество событий с позитивным сюрпризом и валидной 2-дневной доходностью: 8
Медианное 2-дневное процентное изменение после позитивных сюрпризов: 2.86%

(Опционально) Медианное 2-дневное процентное изменение для AMZN за ВСЕ исторические даты: 0.20%
Наблюдение: Медианная 2-дневная доходность в дни позитивных сюрпризов ВЫШЕ, чем медиана за все дни.

--- Дополнительный анализ (Корреляция) ---
Корреляция между величиной 'Surprise (%)' и 2-дневной реакцией цены (для позитивных сюрпризов): 0.4870


## Question 4 Task:
Calculate the median 2-day percentage change in Amazon's stock price following positive earnings surprise days. The 2-day return is defined as `(Close_Price_Day+1 / Close_Price_Day-1 - 1) * 100%`, where "Day 0" is the earnings announcement day (or the next trading day if the announcement is on a non-trading day).

## Analysis Results (using `yf.Ticker("AMZN").earnings_dates`):

*   **Earnings Data Source:** `yf.Ticker("AMZN").earnings_dates`.
*   **Historical AMZN Prices:** Sourced from Yahoo Finance.
*   **Positive Surprise Definition:** Actual EPS > Estimated EPS, OR Surprise (%) > 0 (based on data available via `yfinance`).
*   **Number of Positive Surprise Events with Valid 2-Day Returns:** 8

### Answer to Question 4:
**The median 2-day percentage change in Amazon's stock price following positive earnings surprise days is +2.86%.**

*This result is closest to the answer choice `2.6%` from the provided options (`4.5%`, `3.2%`, `2.6%`, `1.8%`).*

---

### Additional Insights:

1.  **Comparison with Overall Market Dynamics:**
    *   The median 2-day percentage change for AMZN across **all** historical trading days was `+0.17%`.
    *   **Observation:** The median 2-day return on positive surprise days (`+2.86%`) is notably **higher** than the median return for all trading days, suggesting a significant positive market reaction to better-than-expected earnings in the short term.

2.  **Correlation between Surprise Magnitude and Price Reaction:**
    *   The correlation between the "Surprise (%)" magnitude and the 2-day stock price reaction (for positive surprises) was `0.4870`.
    *   **Observation:** This indicates a moderate positive correlation, suggesting that larger positive percentage surprises tend to be associated with a stronger positive stock price reaction. This correlation is higher than observed when using the previous CSV file, potentially due to a different dataset or data quality from `yfinance`.

---

**Conclusion:**
Utilizing earnings calendar data directly from `yfinance`, the analysis reveals that positive earnings surprises for Amazon have historically been followed by a median stock price increase of **+2.86%** over the subsequent 2-day trading period. This effect is substantially greater than the stock's typical daily performance. A moderate positive correlation is also observed between the size of the percentage surprise and the magnitude of the price reaction. The calculated median of `+2.86%` aligns most closely with the `2.6%` option among the provided choices.

In [9]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

def run_earnings_surprise_analysis_yf_calendar():
    ticker_symbol = "AMZN"
    print(f"--- Анализ влияния квартальной отчетности на цену акций {ticker_symbol} (используя yfinance для календаря) ---")

    # --- 1. Загрузка данных об отчетности с помощью yfinance ---
    try:
        amzn_ticker = yf.Ticker(ticker_symbol)
        earnings_df_raw = amzn_ticker.earnings_dates
        if earnings_df_raw is None or earnings_df_raw.empty:
            print(f"Не удалось загрузить данные об отчетности для {ticker_symbol} с помощью yfinance.")
            return
        print("Данные об отчетности загружены с yfinance.")

        earnings_df = earnings_df_raw.reset_index()
        date_col_candidates = ['Earnings Date', 'Date']
        actual_date_col = None
        for col in date_col_candidates:
            if col in earnings_df.columns:
                actual_date_col = col
                break
        
        if not actual_date_col:
            print("ERROR: Не удалось найти столбец с датами отчетности в данных от yfinance.earnings_dates.")
            print(f"Доступные колонки: {earnings_df.columns.tolist()}")
            return

        earnings_df.rename(columns={actual_date_col: 'Earnings DateOriginal'}, inplace=True)
        
        earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings DateOriginal']).dt.tz_localize(None).dt.normalize()
        
        # --- !!! НОВОЕ: Фильтрация по прошедшим датам !!! ---
        current_date_for_filtering = pd.to_datetime(datetime.now().date()).normalize()
        # Для воспроизводимости или анализа на конкретную дату в прошлом, можно раскомментировать и установить:
        # current_date_for_filtering = pd.to_datetime('2024-05-01').normalize() # Пример
        
        print(f"\nФильтрация дат отчетности: оставляем только прошедшие даты (до или равные {current_date_for_filtering.strftime('%Y-%m-%d')}).")
        initial_earnings_dates_count_before_past_filter = len(earnings_df)
        earnings_df = earnings_df[earnings_df['Earnings Date'] <= current_date_for_filtering].copy() 
        print(f"Количество дат отчетности после фильтрации по прошедшим датам: {len(earnings_df)} (было {initial_earnings_dates_count_before_past_filter})")

        if earnings_df.empty:
            print(f"После фильтрации по дате ({current_date_for_filtering.strftime('%Y-%m-%d')}) не осталось прошедших дат отчетности для анализа.")
            return
        # --- КОНЕЦ НОВОЙ ФИЛЬТРАЦИИ ---

        column_map = {
            'EPS Estimate': 'EPS Estimate', 'Reported EPS': 'Reported EPS', 'Surprise(%)': 'Surprise (%)'
        }
        for yf_col, script_col in column_map.items():
            if yf_col in earnings_df.columns:
                earnings_df.rename(columns={yf_col: script_col}, inplace=True)
                if earnings_df[script_col].dtype == 'object':
                    earnings_df[script_col] = earnings_df[script_col].replace(['-', '--', 'N/A', 'NaN', ''], np.nan)
                earnings_df[script_col] = pd.to_numeric(earnings_df[script_col], errors='coerce')
            else:
                if script_col not in earnings_df.columns: 
                    earnings_df[script_col] = np.nan
        
        # Удаляем строки, где дата отчетности оказалась NaN после всех преобразований (маловероятно, но для надежности)
        earnings_df.dropna(subset=['Earnings Date'], inplace=True)
        if earnings_df.empty: # Проверка после dropna, если вдруг все даты были плохими
            print("После обработки не осталось валидных дат отчетности.")
            return


    except Exception as e:
        print(f"ERROR: Ошибка при загрузке или обработке данных об отчетности с yfinance: {e}")
        return

    # --- 2. Загрузка исторических цен акций ---
    print(f"\nЗагрузка исторических цен для {ticker_symbol}...")
    try:
        if earnings_df['Earnings Date'].empty: # Эта проверка теперь еще важнее после фильтрации дат
            print("Нет дат отчетности (после фильтрации по прошедшим датам) для определения диапазона цен.")
            return
        min_hist_date = earnings_df['Earnings Date'].min() - timedelta(days=60)
        max_hist_date = earnings_df['Earnings Date'].max() + timedelta(days=90) # Увеличим запас для T+N дней, особенно T+1 для последнего отчета

        prices_df_multi = yf.download(ticker_symbol, 
                                      start=min_hist_date.strftime('%Y-%m-%d'), 
                                      end=max_hist_date.strftime('%Y-%m-%d'), 
                                      auto_adjust=True, progress=False)
        if prices_df_multi.empty:
            print(f"Не удалось загрузить данные о ценах для {ticker_symbol}.")
            return
        print(f"Данные о ценах {ticker_symbol} загружены.")

        if isinstance(prices_df_multi.columns, pd.MultiIndex):
            if ticker_symbol in prices_df_multi.columns.get_level_values(1):
                prices_df = prices_df_multi.xs(ticker_symbol, level=1, axis=1).copy()
            elif ticker_symbol in prices_df_multi.columns.get_level_values(0): 
                prices_df = prices_df_multi[ticker_symbol].copy()
            else: 
                prices_df = prices_df_multi.copy()
                if isinstance(prices_df.columns, pd.MultiIndex) and 'Close' in prices_df.columns.get_level_values(0):
                     prices_df.columns = prices_df.columns.get_level_values(0)
        else: 
            prices_df = prices_df_multi.copy()
        
        if 'Close' not in prices_df.columns:
             print(f"ERROR: Столбец 'Close' не найден в prices_df. Колонки: {prices_df.columns.tolist()}")
             return
        
        prices_df.index = pd.to_datetime(prices_df.index).tz_localize(None).normalize()
             
    except Exception as e:
        print(f"ERROR: Не удалось загрузить данные о ценах для {ticker_symbol}: {e}")
        return

    # --- 3. Расчет 2-дневного процентного изменения (все даты) ---
    prices_df.loc[:, 'Close_Day1'] = prices_df['Close'].shift(1)
    prices_df.loc[:, 'Close_Day3'] = prices_df['Close'].shift(-1)
    prices_df.loc[:, '2_Day_Return'] = ((prices_df['Close_Day3'] / prices_df['Close_Day1']) - 1) * 100.0
    
    all_2_day_returns = prices_df['2_Day_Return'].copy().dropna()
    
    # --- 4. Идентификация позитивных сюрпризов (согласно условию задачи) ---
    if 'Reported EPS' not in earnings_df.columns or 'EPS Estimate' not in earnings_df.columns:
        print("ERROR: Колонки 'Reported EPS' или 'EPS Estimate' отсутствуют в данных об отчетности (после всех фильтраций).")
        print(f"Доступные колонки в earnings_df: {earnings_df.columns.tolist()}")
        print("Первые строки earnings_df:")
        print(earnings_df.head())
        return

    print("\n--- Диагностика данных отчетности (ПОСЛЕ фильтрации по прошедшим датам, ПЕРЕД фильтрацией на позитивные сюрпризы) ---")
    print(f"Всего дат отчетности (прошедших): {len(earnings_df)}")
    print(f"Количество прошедших дат с 'Reported EPS' (не NaN): {earnings_df['Reported EPS'].notna().sum()}")
    print(f"Количество прошедших дат с 'EPS Estimate' (не NaN): {earnings_df['EPS Estimate'].notna().sum()}")
    
    valid_eps_data_mask = earnings_df['Reported EPS'].notna() & earnings_df['EPS Estimate'].notna()
    print(f"Количество прошедших дат, где ОБА EPS ('Reported' и 'Estimate') присутствуют: {valid_eps_data_mask.sum()}")

    earnings_df['Positive_Surprise'] = False 
    earnings_df.loc[valid_eps_data_mask, 'Positive_Surprise'] = \
        earnings_df.loc[valid_eps_data_mask, 'Reported EPS'] > earnings_df.loc[valid_eps_data_mask, 'EPS Estimate']
    
    print(f"Количество прошедших дат, где Reported EPS > EPS Estimate (среди тех, где оба EPS присутствуют): {earnings_df['Positive_Surprise'].sum()}")
    
    print("\nПример данных отчетности (прошедшие даты) с вычисленным Positive_Surprise (до фильтрации на Positive_Surprise=True):")
    earnings_df_sorted_for_debug = earnings_df.sort_values(by='Earnings Date', ascending=False)
    print(earnings_df_sorted_for_debug[['Earnings Date', 'Reported EPS', 'EPS Estimate', 'Positive_Surprise']].head(40))

    positive_surprise_earnings_df = earnings_df[earnings_df['Positive_Surprise']].copy()

    if positive_surprise_earnings_df.empty:
        print("\nПозитивные сюрпризы (фактический EPS > ожидаемого EPS, оба значения присутствуют, дата прошла) не идентифицированы.")
    else:
        print(f"\nИдентифицировано {len(positive_surprise_earnings_df)} событий с позитивным сюрпризом (Actual EPS > Estimated EPS, оба EPS присутствуют, дата прошла).")

    # --- 5. Сопоставление дат отчетности с торговыми днями и получение доходностей ---
    # (Остальная часть кода остается такой же, как в предыдущем предложенном варианте)
    if not positive_surprise_earnings_df.empty:
        if positive_surprise_earnings_df.index.name != 'Earnings Date':
            positive_surprise_earnings_df = positive_surprise_earnings_df.set_index('Earnings Date')
        positive_surprise_earnings_df.sort_index(inplace=True)
        
        prices_df.sort_index(inplace=True)

        # Проверим, что в prices_df есть данные для merge
        if prices_df.empty or not any(d in prices_df.index for d in positive_surprise_earnings_df.index):
             min_earnings_date = positive_surprise_earnings_df.index.min()
             max_earnings_date = positive_surprise_earnings_df.index.max()
             min_prices_date = prices_df.index.min() if not prices_df.empty else "N/A"
             max_prices_date = prices_df.index.max() if not prices_df.empty else "N/A"
             print(f"WARNING: Диапазон дат отчетности [{min_earnings_date} - {max_earnings_date}] может не пересекаться с диапазоном цен [{min_prices_date} - {max_prices_date}]")


        merged_for_returns = pd.merge_asof(
            left=positive_surprise_earnings_df,
            right=prices_df[['2_Day_Return']], 
            left_index=True, 
            right_index=True, 
            direction='forward', 
            tolerance=pd.Timedelta(days=7) # Увеличим tolerance на всякий случай, если есть длинные выходные/праздники
        )
        
        # Добавим отладочную информацию о merged_for_returns
        # print("\nПервые строки merged_for_returns (после merge_asof):")
        # print(merged_for_returns.head())
        # print("\nПоследние строки merged_for_returns (после merge_asof):")
        # print(merged_for_returns.tail())
        # print(f"Количество строк в merged_for_returns до dropna('2_Day_Return'): {len(merged_for_returns)}")
        # print(f"Количество NaN в '2_Day_Return' в merged_for_returns: {merged_for_returns['2_Day_Return'].isna().sum()}")


        returns_on_surprise_days = merged_for_returns['2_Day_Return'].dropna()
        
        median_return_positive_surprise = None 
        if returns_on_surprise_days.empty:
            print("Не найдено 2-дневных доходностей для дат позитивных сюрпризов после сопоставления с торговыми днями.")
            print("Возможные причины: нет пересечения дат, все 2_Day_Return оказались NaN (например, на краях диапазона цен).")
        else:
            median_return_positive_surprise = returns_on_surprise_days.median()
            print(f"\n--- Результат анализа для {ticker_symbol} ---")
            print(f"Количество событий с позитивным сюрпризом и валидной 2-дневной доходностью: {len(returns_on_surprise_days)}")
            print(f"Ожидаемое количество точек данных по условию: 36 (фактическое для {ticker_symbol} из yfinance может сильно отличаться из-за доступности исторических данных EPS Estimate).")
            print(f"Медианное 2-дневное процентное изменение после позитивных сюрпризов: {median_return_positive_surprise:.2f}%")
    else:
        print("\nАнализ невозможен, так как не найдено позитивных сюрпризов по заданным критериям (прошедшие даты, оба EPS есть, Actual > Estimate).")

    # --- (Опционально) Сравнение со всеми историческими датами ---
    if not all_2_day_returns.empty and 'median_return_positive_surprise' in locals() and median_return_positive_surprise is not None:
        median_return_all_dates = all_2_day_returns.median()
        print(f"\n(Опционально) Медианное 2-дневное процентное изменение для {ticker_symbol} за ВСЕ исторические даты: {median_return_all_dates:.2f}% ({len(all_2_day_returns)} точек)")
        if median_return_positive_surprise > median_return_all_dates:
            print("Наблюдение: Медианная 2-дневная доходность в дни позитивных сюрпризов ВЫШE, чем медиана за все дни.")
        elif median_return_positive_surprise < median_return_all_dates:
            print("Наблюдение: Медианная 2-дневная доходность в дни позитивных сюрпризов НИЖЕ, чем медиана за все дни.")
        else:
            print("Наблюдение: Медианная 2-дневная доходность в дни позитивных сюрпризов РАВНА медиане за все дни.")
    else:
        print("\nНе удалось рассчитать медиану для всех дат или для дней сюрпризов для сравнения.")
    
    # --- (Дополнительно) Корреляция ---
    print("\n--- Дополнительный анализ (Корреляция с Surprise (%)) ---")
    surprise_col_name = 'Surprise (%)' 
    # Проверяем, что merged_for_returns был создан и не пуст
    if 'merged_for_returns' in locals() and not merged_for_returns.empty and not positive_surprise_earnings_df.empty :
        # Также убедимся, что колонка Surprise (%) существует в merged_for_returns (она должна прийти из positive_surprise_earnings_df)
        if surprise_col_name in merged_for_returns.columns and \
           pd.api.types.is_numeric_dtype(merged_for_returns[surprise_col_name]) and \
           '2_Day_Return' in merged_for_returns.columns and \
           merged_for_returns['2_Day_Return'].notna().any():

            valid_corr_data = merged_for_returns[[surprise_col_name, '2_Day_Return']].dropna()
            
            if len(valid_corr_data) > 1: 
                correlation = valid_corr_data[surprise_col_name].corr(valid_corr_data['2_Day_Return'])
                print(f"Корреляция между величиной '{surprise_col_name}' и 2-дневной реакцией цены (для позитивных сюрпризов по EPS): {correlation:.4f}")
                print(f"Количество точек данных для корреляции: {len(valid_corr_data)}")
            else:
                print(f"Недостаточно валидных данных ({len(valid_corr_data)}) для расчета корреляции по '{surprise_col_name}'.")
        else:
            print(f"Не удалось рассчитать корреляцию: отсутствуют необходимые числовые данные ('{surprise_col_name}', '2_Day_Return') или все значения доходности NaN.")
            if surprise_col_name not in merged_for_returns.columns:
                 print(f"  Колонка '{surprise_col_name}' отсутствует в merged_for_returns (возможно, ее не было в earnings_df или она была отброшена).")
            elif not pd.api.types.is_numeric_dtype(merged_for_returns[surprise_col_name]):
                 print(f"  Колонка '{surprise_col_name}' не числового типа: {merged_for_returns[surprise_col_name].dtype}")
            if '2_Day_Return' not in merged_for_returns.columns:
                 print(f"  Колонка '2_Day_Return' отсутствует в merged_for_returns.")
            elif not merged_for_returns['2_Day_Return'].notna().any():
                 print(f"  Все значения в '2_Day_Return' являются NaN.")
    else:
        print("Невозможно выполнить анализ корреляции: нет данных о сюрпризах с доходностями (merged_for_returns не был создан или пуст).")

if __name__ == '__main__':
    run_earnings_surprise_analysis_yf_calendar()

--- Анализ влияния квартальной отчетности на цену акций AMZN (используя yfinance для календаря) ---
Данные об отчетности загружены с yfinance.

Фильтрация дат отчетности: оставляем только прошедшие даты (до или равные 2025-06-07).
Количество дат отчетности после фильтрации по прошедшим датам: 8 (было 12)

Загрузка исторических цен для AMZN...
Данные о ценах AMZN загружены.

--- Диагностика данных отчетности (ПОСЛЕ фильтрации по прошедшим датам, ПЕРЕД фильтрацией на позитивные сюрпризы) ---
Всего дат отчетности (прошедших): 8
Количество прошедших дат с 'Reported EPS' (не NaN): 8
Количество прошедших дат с 'EPS Estimate' (не NaN): 8
Количество прошедших дат, где ОБА EPS ('Reported' и 'Estimate') присутствуют: 8
Количество прошедших дат, где Reported EPS > EPS Estimate (среди тех, где оба EPS присутствуют): 8

Пример данных отчетности (прошедшие даты) с вычисленным Positive_Surprise (до фильтрации на Positive_Surprise=True):
   Earnings Date  Reported EPS  EPS Estimate  Positive_Surprise


In [14]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta

def run_earnings_surprise_analysis_yf_calendar():
    ticker_symbol = "AMZN"
    print(f"--- Анализ влияния квартальной отчетности на цену акций {ticker_symbol} (используя yfinance для календаря) ---")

    # --- 1. Загрузка данных об отчетности ---
    try:
        amzn_ticker = yf.Ticker(ticker_symbol)
        earnings_df_raw = amzn_ticker.earnings_dates
        # ... (остальная часть загрузки earnings_df без изменений, как в предыдущих версиях, где было 8 точек) ...
        if earnings_df_raw is None or earnings_df_raw.empty:
            print(f"Не удалось загрузить данные об отчетности для {ticker_symbol}.")
            return
        print("Данные об отчетности загружены с yfinance.")
        earnings_df = earnings_df_raw.reset_index()
        date_col_candidates = ['Earnings Date', 'Date']
        actual_date_col = None
        for col in date_col_candidates:
            if col in earnings_df.columns:
                actual_date_col = col
                break
        if not actual_date_col:
            print("ERROR: Не удалось найти столбец с датами отчетности.")
            return
        earnings_df.rename(columns={actual_date_col: 'Earnings DateOriginal'}, inplace=True)
        earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings DateOriginal']).dt.tz_localize(None).dt.normalize()
        column_map = {
            'EPS Estimate': 'EPS Estimate', 'Reported EPS': 'Reported EPS', 'Surprise(%)': 'Surprise (%)'
        }
        for yf_col, script_col in column_map.items():
            if yf_col in earnings_df.columns:
                earnings_df.rename(columns={yf_col: script_col}, inplace=True)
                if earnings_df[script_col].dtype == 'object':
                    earnings_df[script_col] = earnings_df[script_col].replace(['-', '--', 'N/A', 'NaN', ''], np.nan)
                earnings_df[script_col] = pd.to_numeric(earnings_df[script_col], errors='coerce')
            else:
                if script_col not in earnings_df.columns: 
                    earnings_df[script_col] = np.nan
        earnings_df.dropna(subset=['Earnings Date'], inplace=True)
        if earnings_df.empty:
            print("После обработки не осталось валидных дат отчетности.")
            return
    except Exception as e:
        print(f"ERROR при загрузке данных отчетности: {e}")
        return

    # --- 2. Загрузка исторических цен акций ---
    print(f"\nЗагрузка исторических цен для {ticker_symbol}...")
    try:
        if earnings_df['Earnings Date'].empty:
            print("Нет дат отчетности для диапазона цен.")
            return
        min_date_for_prices = earnings_df['Earnings Date'].min() - timedelta(days=60)
        max_date_for_prices = earnings_df['Earnings Date'].max() + timedelta(days=90)
        prices_df_multi = yf.download(ticker_symbol, 
                                      start=min_date_for_prices.strftime('%Y-%m-%d'), 
                                      end=max_date_for_prices.strftime('%Y-%m-%d'), 
                                      auto_adjust=True, progress=False)
        # ... (остальная часть загрузки prices_df без изменений) ...
        if prices_df_multi.empty:
            print(f"Не удалось загрузить цены для {ticker_symbol}.")
            return
        print(f"Данные о ценах {ticker_symbol} загружены.")
        if isinstance(prices_df_multi.columns, pd.MultiIndex):
            if ticker_symbol in prices_df_multi.columns.get_level_values(1): prices_df = prices_df_multi.xs(ticker_symbol, level=1, axis=1).copy()
            elif ticker_symbol in prices_df_multi.columns.get_level_values(0): prices_df = prices_df_multi[ticker_symbol].copy()
            else: 
                prices_df = prices_df_multi.copy()
                if isinstance(prices_df.columns, pd.MultiIndex) and 'Close' in prices_df.columns.get_level_values(0): prices_df.columns = prices_df.columns.get_level_values(0)
        else: prices_df = prices_df_multi.copy()
        if 'Close' not in prices_df.columns:
             print(f"ERROR: 'Close' не найден в prices_df.")
             return
        prices_df.index = pd.to_datetime(prices_df.index).tz_localize(None).normalize()
    except Exception as e:
        print(f"ERROR при загрузке цен: {e}")
        return

    # --- 3. Расчет 2-дневного процентного изменения (Close[T+1] / Close[T-1]) - 1 ---
    print("\nРасчет доходности по формуле: (Close[T+1] / Close[T-1]) - 1")
    prices_df.loc[:, 'Close_Day1'] = prices_df['Close'].shift(1)  # Цена закрытия T-1
    prices_df.loc[:, 'Close_Day3'] = prices_df['Close'].shift(-1) # Цена закрытия T+1
    prices_df.loc[:, '2_Day_Return'] = ((prices_df['Close_Day3'] / prices_df['Close_Day1']) - 1) * 100.0
    
    # --- 4. Идентификация позитивных сюрпризов ---
    # ... (логика идентификации positive_surprise_earnings_df без изменений, должна дать 8 точек) ...
    if 'Reported EPS' not in earnings_df.columns or 'EPS Estimate' not in earnings_df.columns:
        print("ERROR: Колонки 'Reported EPS' или 'EPS Estimate' отсутствуют.")
        return
    print("\n--- Диагностика данных отчетности (ПЕРЕД фильтрацией на позитивные сюрпризы) ---")
    print(f"Всего дат отчетности из yfinance (до фильтрации по EPS): {len(earnings_df)}")
    valid_eps_data_mask = earnings_df['Reported EPS'].notna() & earnings_df['EPS Estimate'].notna()
    print(f"Количество дат, где ОБА EPS ('Reported' и 'Estimate') присутствуют: {valid_eps_data_mask.sum()}")
    earnings_df['Positive_Surprise'] = False 
    earnings_df.loc[valid_eps_data_mask, 'Positive_Surprise'] = \
        earnings_df.loc[valid_eps_data_mask, 'Reported EPS'] > earnings_df.loc[valid_eps_data_mask, 'EPS Estimate']
    print(f"Количество дат, где Reported EPS > EPS Estimate (среди тех, где оба EPS присутствуют): {earnings_df['Positive_Surprise'].sum()}")
    positive_surprise_earnings_df = earnings_df[earnings_df['Positive_Surprise']].copy()


    # --- 5. Сопоставление с доходностями (ИСПОЛЬЗУЕМ ВСЕ ТОЧКИ) ---
    if not positive_surprise_earnings_df.empty:
        if positive_surprise_earnings_df.index.name != 'Earnings Date':
            positive_surprise_earnings_df = positive_surprise_earnings_df.set_index('Earnings Date')
        positive_surprise_earnings_df.sort_index(inplace=True)

        print(f"\nВсего идентифицировано {len(positive_surprise_earnings_df)} событий с позитивным сюрпризом.")
        print(f"Даты этих событий (от ранних к поздним): {positive_surprise_earnings_df.index.strftime('%Y-%m-%d').tolist()}")
        
        # Используем все доступные точки
        selected_earnings_df_for_analysis = positive_surprise_earnings_df.copy()
        print(f"Используются ВСЕ {len(selected_earnings_df_for_analysis)} доступные даты для расчета медианы.")
        
        prices_df.sort_index(inplace=True)
        merged_for_returns = pd.merge_asof(
            left=selected_earnings_df_for_analysis,
            right=prices_df[['2_Day_Return']], # Используем '2_Day_Return'
            left_index=True, 
            right_index=True, 
            direction='forward', 
            tolerance=pd.Timedelta(days=7) 
        )
        
        returns_on_surprise_days = merged_for_returns['2_Day_Return'].dropna() # Используем '2_Day_Return'
        
        print("\nДоходности (2_Day_Return) для ВСЕХ 8 дат (после merge_asof и dropna):")
        print(merged_for_returns[merged_for_returns['2_Day_Return'].notna()][['Reported EPS', 'EPS Estimate', '2_Day_Return']])


        median_return_positive_surprise = None 
        if returns_on_surprise_days.empty:
            print("Не найдено доходностей (2_Day_Return).")
        else:
            median_return_positive_surprise = returns_on_surprise_days.median()
            print(f"\n--- Результат анализа для {ticker_symbol} (для ВСЕХ {len(returns_on_surprise_days)} точек, формула Close[T+1]/Close[T-1]) ---")
            print(f"Количество событий (точек данных) для расчета медианы: {len(returns_on_surprise_days)}")
            print(f"Медианное 2-дневное процентное изменение (T+1 vs T-1) после позитивных сюрпризов: {median_return_positive_surprise:.2f}%")
    else:
        print("\nПозитивные сюрпризы не идентифицированы, анализ невозможен.")

    # ...

if __name__ == '__main__':
    run_earnings_surprise_analysis_yf_calendar()

--- Анализ влияния квартальной отчетности на цену акций AMZN (используя yfinance для календаря) ---
Данные об отчетности загружены с yfinance.

Загрузка исторических цен для AMZN...
Данные о ценах AMZN загружены.

Расчет доходности по формуле: (Close[T+1] / Close[T-1]) - 1

--- Диагностика данных отчетности (ПЕРЕД фильтрацией на позитивные сюрпризы) ---
Всего дат отчетности из yfinance (до фильтрации по EPS): 12
Количество дат, где ОБА EPS ('Reported' и 'Estimate') присутствуют: 8
Количество дат, где Reported EPS > EPS Estimate (среди тех, где оба EPS присутствуют): 8

Всего идентифицировано 8 событий с позитивным сюрпризом.
Даты этих событий (от ранних к поздним): ['2023-08-03', '2023-10-26', '2024-02-01', '2024-04-30', '2024-08-01', '2024-10-31', '2025-02-06', '2025-05-01']
Используются ВСЕ 8 доступные даты для расчета медианы.

Доходности (2_Day_Return) для ВСЕХ 8 дат (после merge_asof и dropna):
               Reported EPS  EPS Estimate  2_Day_Return
Earnings Date                  